In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
school_support = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\School Support.csv")
enrollment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Enrollment.csv")
corporal_punishment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Corporal Punishment.csv")
expulsions = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Expulsions.csv")

C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_5832\1544074868.py:1: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  school_support = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\School Support.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_5832\1544074868.py:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  enrollment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Enrollment.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_5832\1544074868.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  corporal_punishment = pd.read_csv("C:\\Users\Omkaar Sampigeadi\Downloads\DataNova Datasets\\2021-22-crdc-data\SCH\Corporal Punishment.csv")
C:\Users\Omkaar Sampigeadi\AppData\Local\Temp\ipykernel_5832\1

In [20]:
print("SCH_ENR_504_F" in enrollment)

True


In [22]:
# Descriptive statistics for key metrics
def analyze_descriptive_stats(df, metric_columns, table_heading):
    """Generate descriptive statistics for specified columns"""
    stats_summary = df[metric_columns].describe()
    print(f"Descriptive Statistics: {table_heading}")
    print(stats_summary)
    print()


# Example for school support data
support_metrics = ['SCH_FTETEACH_TOT', 'SCH_FTETEACH_CERT', 'SCH_FTETEACH_NOTCERT']
if all(col in school_support.columns for col in support_metrics):
   analyze_descriptive_stats(school_support, support_metrics, "School Support Metrics")


# Example for enrollment data - focusing on disability-related columns
disability_metrics = [
    'SCH_ENR_IDEA_M',     # Male students served under IDEA (Individuals with Disabilities Education Act)
    'SCH_ENR_IDEA_F',     # Female students served under IDEA (Individuals with Disabilities Education Act
    'SCH_ENR_504_M',      # Male students with 504 plans (Section 504 of the Rehabilitation Act)
    'SCH_ENR_504_F'       # Female students with 504 plans (Section 504 of the Rehabilitation Act)
]

analyze_descriptive_stats(enrollment, disability_metrics, "Enrollment Metrics for Students with Disabilities")

# First 5 columns

Descriptive Statistics: School Support Metrics
       SCH_FTETEACH_TOT  SCH_FTETEACH_CERT  SCH_FTETEACH_NOTCERT
count      98010.000000       98010.000000          98010.000000
mean          34.605761          33.161289              1.442351
std           27.210973          26.455871              4.497697
min           -6.000000          -6.000000             -6.000000
25%           18.010000          17.000000              0.000000
50%           29.330000          28.000000              0.000000
75%           43.430000          42.000000              1.000000
max         1074.000000         784.000000            305.000000

Descriptive Statistics: Enrollment Metrics for Students with Disabilities
       SCH_ENR_IDEA_M  SCH_ENR_IDEA_F  SCH_ENR_504_M  SCH_ENR_504_F
count    98010.000000    98010.000000   98010.000000   98010.000000
mean        44.636700       23.561912       7.944904       5.245975
std         44.503713       25.511122      19.941082      16.584246
min         -9.000000